In [ ]:
!nvcc --version  # correct double dash


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-pka_yq4r
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-pka_yq4r
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=fdb30b442a03a2b978d94f242f4f3839c3b517111d4c5bcc31496f40b5aacbba
  Stored in directory: /tmp/pip-ephem-wheel-cache-rpvsjvcr/wheels/ef/1d/c6/f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter


Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpy4hhxxr2".


In [ ]:
code = r'''
#include <stdio.h>
#include <cuda.h>

__global__ void minReduce(int* input, int n) {
    int tid = threadIdx.x;
    int step_size = 1;
    int num_threads = blockDim.x;

    while (num_threads > 0) {
        int fst = tid * step_size * 2;
        int snd = fst + step_size;

        if (tid < num_threads && snd < n) {
            if (input[fst] > input[snd]) {
                input[fst] = input[snd];
            }
        }

        __syncthreads();
        step_size <<= 1;
        num_threads >>= 1;
    }
}

int main() {
    const int count = 8;
    const int size = count * sizeof(int);
    int h[count] = {13, 65, 15, 14, 33, 23, 30, 4};

    int* d;
    cudaMalloc(&d, size);
    cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

    minReduce<<<1, count / 2>>>(d, count);

    int result;
    cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);
    printf("Smallest number is: %d\n", result);

    cudaFree(d);
    return 0;
}
'''

with open('min_reduce.cu', 'w') as f:
    f.write(code)


In [ ]:
!nvcc min_reduce.cu -o min_reduce


In [ ]:
!./min_reduce


Smallest number is: 4
